# Introduction

B. McDonough \
J.Cui \
G. Marous

### Contents
1. What is a qumode?
2. Importing Bosonic Qiskit and making circuits containing Qumodes
3. Physical interpretation of a qumode
4. Interaction between qumodes and qubits


## What is a qumode?

Bits are the world's simplest alphabet, allowing any piece of information to be represented using the characters 0 and 1. Qubits are an extension of the bit to quantum mechanics, allowing us to encode quantum information using arbitrary superpositions of the characters $|0\rangle$ and $|1\rangle$, formally known as *basis states*. But what if our alphabet is constructed from not just two characters (i.e., two basis states), but a countably infinite number? But what if instead of two characters, the alphabet were infinite?

A bosonic mode, or "qumode," is an object that can host any number of quanta. This endows the qumode with an infinite "alphabet" of basis states, each identified by the number of quanta corresponding to that state, i.e. $|0\rangle, |1\rangle, |2\rangle, |3\rangle,...$, all the way to infinity. These states are formally referred to as "Fock" states.

![Alt desc](https://github.com/benmcdonough20/bosonic-qiskit-tutorials/raw/main/bosonic_qiskit_textbook/figures/energy_diagram.png)

The figure above shows the two distinct states of a qubit (a) compared with the infinitely many distinguishable states of a qumode (b). The energies of the qumode states are evenly spaced because each photon contributes equally to the total energy.

## Dirac notation
Dirac notation is a convenient way to represent quantum states, denoting a state by placing a label for the state within a "ket"--$|$ and $\rangle$. Mutually exclusive, or "distinguishable" states, such as states with different energies or a different number of photons, are assigned to orthogonal vectors. This allows us to easily go back and forth between matrix notation and Dirac notation. For example, using the mutually exclusive states $|0\rangle$ and $|1\rangle$, 
$$
\alpha |0\rangle + \beta |1 \rangle \ \dot{=} \ \alpha \begin{pmatrix} 1 \\ 0 \end{pmatrix} + \beta \begin{pmatrix} 0 \\ 1 \end{pmatrix} = \begin{pmatrix} \alpha \\ \beta \end{pmatrix}
$$
The $\dot =$ sign, meaning "represented by," is used instead of equality to indicate that the choice of orthogonal vectors to assign to each state is arbitrary. We could easily have assigned $|0 \rangle$ to $\begin{pmatrix} 0 \\ 1\end{pmatrix}$ and $|1\rangle$ to $\begin{pmatrix} 1 \\ 0 \end{pmatrix}$, resulting in a different representation.

Dirac notation becomes particularly important for describing qumodes. With an infinite number of mutually exclusive states, the vectors would be infinitely long! The most general state $|\psi\rangle$ of a qumode can be written in the number basis as
$$
|\psi\rangle = \sum_{n=0}^\infty a_n |n\rangle
$$
where $a_n$ are complex numbers.

## From qubits to qumodes
Most of the familiar axioms for qubits generalize directly to qumodes. These are, briefly,
1. **The Borne Rule:** If $|\psi\rangle$ is the state of the qumode and $\{|n\rangle\}$ is an orthonormal basis, then the probability of measuring $|\psi\rangle$ in the state $|n\rangle$ is
$$
p_{\psi}(n) = |\langle n | \psi \rangle|^2
$$

2. **Mutually exclusive states are orthogonal:** if $m\neq n$, then $|n\rangle$ and $|m\rangle$ correspond to a mode with two different photon numbers, making these states mutually exclusive.  The Borne rule implies that the states are orthogonal, meaning that their *inner product* vanishes: $\langle m | n \rangle = 0$. 

3. **Normalization:** The Borne rule also implies that all physical quantum states should be normalized. In the case of a discrete basis like the Fock basis, this implies that $\langle n | m \rangle = \delta_{mn}$. Kets such as $|x\rangle$ comprising a continuous basis generally do not represent physical states, and they are delta-function normalized, meaning that $\langle x' | x \rangle = \delta(x-x')$. Learn more about this in `operations.ipynb`.

## Importing Bosonic Qiskit
Bosonic Qiskit is a C2QA project extending the functionality of Qiskit to simulate hybrid qumode-qubit systems. For more information about installation and usage, visit the [Github repository](https://github.com/C2QA/bosonic-qiskit). A [set of tutorials](https://github.com/C2QA/bosonic-qiskit/tree/main/tutorials) is also available as part of this repository. For a fairly light introduction, refer to [this article on Medium](
https://medium.com/qiskit/introducing-bosonic-qiskit-a-package-for-simulating-bosonic-and-hybrid-qubit-bosonic-circuits-1e1e528287bb). For a more succinct and technical description of the functionality of Bosonic Qiskit, read the paper [here](https://arxiv.org/abs/2209.11153).

In [ ]:
# To use the package locally, add the C2QA repository's root folder to the path prior to importing c2qa.
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Cheat to get MS Visual Studio Code Jupyter server to recognize Python venv
module_path = os.path.abspath(os.path.join("../../venv/Lib/site-packages"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [1]:
import c2qa
import qiskit
import numpy as np

### Creating a Qumode

Bosonic Qiskit extends Qiskit by approximating the infinite states of a *qumode* by using multiple qubits. The `QumodeRegister` is an abstraction of the `QuantumRegister` object designed to represent qumodes. The `num_qubits_per_qumode` parameter specifies the number of qubits $k$ used to represent the first $2^k$ levels of a qumode.

The following piece of code creates a qumode using 4 qubits. Using too few qubits to represent a qumode may result in an inaccurate simulation due to truncation errors. It is the responsibility  of the user to identify the number of simulation qubits for reasonable performance without sacrificing accuracy. 

In [2]:
qmr = c2qa.QumodeRegister(
    num_qumodes=1,
    num_qubits_per_qumode=4
)

The Bosonic extension of Qiskit's `QuantumCircuit` is the `CVCircuit`, which is used to initialize, add sequential instructions, and perform readout of a qumode.

In [3]:
circuit = c2qa.CVCircuit(qmr)

## Photons in a cavity
Qumodes can be implemented, for example, using the electromagnetic modes of a cavity, which is like a box with mirrored walls. Photons can be added to and removed from the box, and the reflective walls of the box give it the ability to trap photons inside. Quantum mechanics tells us that photons are not just particles, but also electromagnetic waves. In the empty space inside the cavity, Maxwell's equations can be used to show that each component of the electric field in the cavity satisfies the equation
$$
\vec E(x,y,z,t) = \vec E_0 \operatorname{Re}(e^{i(k_x x+k_yy+k_zz - \omega t)})
$$
where $\vec E_0$ is a constant and $\omega^2 = c^2(k_x^2+k_y^2+k_z^2)$ is a frequency. Maxwell's equations also require that any transverse component of the field vanishes at the walls, which gives the following conditions on $k_x, k_y, k_z$:
\begin{align*}
E(x, y, z) &= E(x+n_xL_x, y, z) &\implies k_x &= \frac{2\pi n_x}{L_x}\\
E(x, y, z) &= E(x, y+n_yL_y, z) &\implies k_y &= \frac{2\pi n_y}{L_y}\\
E(x, y, z) &= E(x, y, z+n_zL_z) &\implies k_z &= \frac{2\pi n_z}{L_z}
\end{align*}
For some integers $n_x, n_y, n_z$. This gives a set of discrete solutions for the allowed frequencies of photons in the cavity:
$$
\omega_{n_x, n_y, n_z}^2 = (2\pi c)^2\left(\frac{n_x^2}{L_x^2}+\frac{n_y^2}{L_y^2}+\frac{n_z^2}{L_z^2}\right)
$$
The cavity can only store photons with frequencies at these specific increments. This is illustrated pictorially for a 1D cavity in the diagram below:

![A cavity can only store photons with a discrete set of frequencies determined by the boundary conditions on the electric field](https://github.com/benmcdonough20/bosonic-qiskit-tutorials/raw/main/bosonic_qiskit_textbook/figures/qumodes.png)

The energy of a single photon is related to its angular frequency by $\varepsilon = \omega \hbar$. Thus, each allowed frequency $\omega$, or "mode" of the cavity, corresponds to an energy $\hbar\omega N$, where $N$ is the number of photons in that particular mode. Each of these cavity modes is an example of a qumode. This is certainly not the only way to implement qumodes. For further information, the connection between qumodes and harmonic oscillators is explored in `operations.ipynb`.

### Initializing Fock states

A Fock state is a state of the qumode with a definite number of photons. The `cv_initialize` method is used to initialize the qumode to different states. The following piece of code initializes the qumode to Fock state $|{0}\rangle$ (Referring back to the introduction, the number in the ket refers to the number of photons corresponding to that state). The first argument refers to which Fock state you want to initialize, and the second argument references the qumode in the qumode register. The Fock number cannot be higher than $2^k-1$ for this qumode, where $k$ is the number of simulation qubits per qumode.

In [4]:
circuit.cv_initialize(0, qmr[0])

The method `cv_initialize` can also create *superpositions* of Fock states. Instead of an integer for the first argument, the method can take a list of the coefficients for each Fock state, with the $i^{th}$ coefficient corresponding to Fock state $|i\rangle$. The following piece of code initilizes the qumode to $\frac{|{0}\rangle+|{1}\rangle}{\sqrt{2}}$.

In [5]:
circuit.cv_initialize([1/np.sqrt(2),1/np.sqrt(2)], qmr[0])

## How do qubits and bosonic modes interact?
At the heart of quantum computing with bosonic modes is the ability to make a qubit and a qumode interact. Just like both Clifford and non-Clifford gates are necessary for universal control of a qubit, both *Gaussian* and *non-Gaussian* gates are necessary to control a qumode. For qubits that are not truly two-level systems, an anharmonic potential is necessary to isolate the first two energy levels. Non-Gaussian gates on a qumode are only possible because of the anharmonicity introduced by the qubit. The following shows what is meant by "anharmonicity." The figure below shows the cosine potential and first five energy levels of a transmon qubit vs the quadratic potential and first five energy levels of a qumode. The difference between the first two energy levels is chosen to be the same on both. The quadratic potential of the qumode leads to evenly spaced energy levels, whereas the levels of the transmon are not evenly spaced. This plot was produced using [scQubits](https://scqubits.readthedocs.io/en/latest/).

![Energy levels of a transmon compared to an oscillator](https://raw.githubusercontent.com/benmcdonough20/bosonic-qiskit-tutorials/main/bosonic_qiskit_textbook/figures/tmon.svg)

The physicist's prototype for describing qubit-qumode coupling is cavity quantum electrodynamics, which at its most basic level consists of a two-level atom embedded within a cavity. This is illustrated in the figure below, reproduced from [Schoelkopf et al.](https://www.nature.com/articles/451664a)

![Diagram of a prototypical cQED system](https://github.com/benmcdonough20/bosonic-qiskit-tutorials/raw/main/bosonic_qiskit_textbook/figures/cQED.png)

The atom interacts with the cavity by emitting a photon at the frequency of one of the modes of the cavity, which is reflected by the cavity and reabsorbed by the atom. The interaction splits the energy levels so that the state of the cavity can be entangled with the state of the qubit, as illustrated in the following diagram reproduced from Fig. 9 in [Blais et al.](https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.93.025005)


![Energy levels of the Jaynes-Cummings Hamiltonina](https://github.com/benmcdonough20/bosonic-qiskit-tutorials/raw/main/bosonic_qiskit_textbook/figures/JCshift.png)

The state of the qubit is labeled as $|g\rangle$ (ground state) or $|e\rangle$ (first excited state), and the Fock states of the qumode are labeled numerically. The gray bars show the energy levels of the combined qumode-qubit system without considering the coupling, which depend only on the total number of photons in the system. The coupling of the atom to a single mode of the cavity lifts this degeneracy. In the so-called strong-dispersive coupling regime (for more info see Blais et al), this splitting enables gates on the cavity mode which are conditioned on the state of the qubit.

## Combining Qumodes and Qubits
The following piece of code demonstrates how a CVCircuit — a new subclass of the Qiskit QuantumCircuit class —  can support qumodes, qubits, and classical registers.

In [6]:
qmr = c2qa.QumodeRegister(
    num_qumodes=1,
    num_qubits_per_qumode=4
)

qr = qiskit.QuantumRegister(
    size=1
)

cr = qiskit.ClassicalRegister(
    size = 1
)

circuit = c2qa.CVCircuit(qmr, qr, cr)

## Citations
[1] Stavenger, Timothy J., Eleanor Crane, Kevin Smith, Christopher T. Kang, Steven M. Girvin, and Nathan Wiebe. 2022. “Bosonic Qiskit.” arXiv. https://doi.org/10.48550/arXiv.2209.11153.

[2] Schoelkopf, R. J., and S. M. Girvin. 2008. “Wiring up Quantum Systems.” Nature 451 (7179): 664–69. https://doi.org/10.1038/451664a.

[3] Blais, Alexandre, Arne L. Grimsmo, S. M. Girvin, and Andreas Wallraff. 2021. “Circuit Quantum Electrodynamics.” Reviews of Modern Physics 93 (2): 025005. https://doi.org/10.1103/RevModPhys.93.025005.
